In [1]:
import numpy as np
import tensorflow as tf
#import maxout
import highway_maxout as hmn
import utils
import encoder as enc
import dataset as ds
import train as tr
import decoder as dec
import time
from itertools import islice


In [2]:
#======= FLAGS ==========
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('maxout_layer_size', 200, 'Maxout layer size')
tf.app.flags.DEFINE_integer('max_sequence_length', 160, 'Max length of context')
tf.app.flags.DEFINE_integer('max_question_length', 40, 'Max question tokens length')
tf.app.flags.DEFINE_float('learning_rate', 0.001, 'Learning Rate')
tf.app.flags.DEFINE_integer('maxout_pooling_size', 8, 'Maxout pooling size')
tf.app.flags.DEFINE_integer('lstm_size', 200, 'LSTM cell internal size')
tf.app.flags.DEFINE_string('log_path', '/tmp/working/logs', 'logs location')
tf.app.flags.DEFINE_integer('acc_batch_size', 10, 'How many examples to use to calculate accuracy')
tf.app.flags.DEFINE_integer('train_batch_size', 20, 'Train Batch Size')
tf.app.flags.DEFINE_integer('max_decoder_iterations', 4, 'Decoder Iterations')
tf.app.flags.DEFINE_integer('max_epoch', 100, 'Max Train Epoch Count')


In [3]:
# remove all variables
#tf.reset_default_graph();

lstm_size = FLAGS.lstm_size
acc_batch_size = FLAGS.acc_batch_size
word_vector_size = 300
maxout_pooling_size = FLAGS.maxout_pooling_size
max_decoder_iterations = FLAGS.max_decoder_iterations
maxout_layer_size = FLAGS.maxout_layer_size;
max_epoch = FLAGS.max_epoch;
max_sequence_length = FLAGS.max_sequence_length;
max_question_length = FLAGS.max_question_length
batch_size = FLAGS.train_batch_size


dropout_rate_ph = tf.placeholder(tf.float32)
question_ph = tf.placeholder(tf.float32, [batch_size, max_question_length, word_vector_size], name="q_input")
document_ph = tf.placeholder(tf.float32, [batch_size, max_sequence_length, word_vector_size], name="d_input")
doc_len_ph = tf.placeholder(tf.int32, [batch_size])
que_len_ph = tf.placeholder(tf.int32, [batch_size])
document_size = doc_len_ph
question_size = que_len_ph

with tf.name_scope('ENCODER'):
    # LSTM cell initialization
    lstm = tf.nn.rnn_cell.LSTMCell(lstm_size)
    lstm = tf.nn.rnn_cell.DropoutWrapper(cell=lstm, output_keep_prob=dropout_rate_ph)


# LSTM cells for Bi-LSTM for COATINATION ENCODER
with tf.name_scope('COATTENTION_ENCODER'):
    lstm_cenc_fw = tf.nn.rnn_cell.LSTMCell(lstm_size)
    lstm_cenc_fw = tf.nn.rnn_cell.DropoutWrapper(cell=lstm_cenc_fw, output_keep_prob=dropout_rate_ph)
    lstm_cenc_bw = tf.nn.rnn_cell.LSTMCell(lstm_size)
    lstm_cenc_bw = tf.nn.rnn_cell.DropoutWrapper(cell=lstm_cenc_bw, output_keep_prob=dropout_rate_ph)

# create lstm cell for DYNAMIC POINTING DECODER
lstm_dec = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# get lstm initial state of zeroes
#lstm_dec_state = lstm_dec.zero_state(1, tf.float32)
start_pos = 0; # ?generate random between (0, document_size-1)
end_pos = 0;   # ?generate random between (0, document_size-1)

# create sentinel vector variable for both encodings 
#with tf.variable_scope("scope1") as scope:
sentinel_q = tf.get_variable("sentinel_q", [ 1, lstm_size ], initializer = tf.random_normal_initializer())
sentinel_d = tf.get_variable("sentinel_d", [ 1, lstm_size ], initializer = tf.random_normal_initializer()) 

tf.summary.histogram('sentinel_q', sentinel_q)
tf.summary.histogram('sentinel_q_max', tf.reduce_max(sentinel_q))
tf.summary.histogram('sentinel_d', sentinel_d)
tf.summary.histogram('sentinel_d_max', tf.reduce_max(sentinel_d))

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

In [4]:
# (batch, D, 2L)
U = enc.encoderBatch(
    document_ph, question_ph, 
    document_size, question_size, 
    lstm, lstm_cenc_fw, lstm_cenc_bw, 
    sentinel_d, sentinel_q, 
    FLAGS)
print(U)

Tensor("COATTENTION_ENCODER_1/Slice_2:0", shape=(20, 160, 400), dtype=float32)


In [5]:
# ===================== DYNAMIC POINTING DECODER =============

sum_start_scores, sum_end_scores = dec.decoderBatch(U, lstm_dec, dropout_rate_ph, FLAGS)

start_true = tf.placeholder(tf.int32, [batch_size]);
end_true   = tf.placeholder(tf.int32, [batch_size]);
    
#sum_start_scores = tf.Print(sum_start_scores, [start_end_true, sum_start_scores], 'sum_start_scores: ')
#sum_end_scores = tf.Print(sum_end_scores, [start_end_true, sum_end_scores], 'sum_start_scores: ')
    
    
# loss and train step
onehot_labels_start = tf.one_hot(start_true, max_sequence_length)
onehot_labels_end   = tf.one_hot(end_true, max_sequence_length)
#print("sum_start_scores", sum_start_scores)
with tf.name_scope('Loss'):
    loss_start = tf.nn.softmax_cross_entropy_with_logits(
        labels = onehot_labels_start,
        logits = sum_start_scores)
    loss_start = tf.reduce_mean(loss_start)
    loss_end = tf.nn.softmax_cross_entropy_with_logits(
        labels = onehot_labels_end,
        logits = sum_end_scores)
    loss_end = tf.reduce_mean(loss_end)
    sum_loss = loss_start + loss_end


tf.summary.histogram('sum_start_scores', sum_start_scores)
tf.summary.histogram('sum_end_scores', sum_end_scores)
tf.summary.scalar('loss_test', sum_loss, ["TEST_STAT"])
tf.summary.scalar('loss_train', sum_loss, ["TRAIN_STAT"])
    
with tf.name_scope('Accuracy'):
    with tf.name_scope('Prediction'):
        pr_start_idx = tf.to_int32(tf.argmax(sum_start_scores, 1))
        pr_end_idx = tf.to_int32(tf.argmax(sum_end_scores, 1))
         
    with tf.name_scope('Accuracy'):
        accuracy_avg = tf.py_func(utils.f1_score_int_avg, [pr_start_idx, pr_end_idx, start_true, end_true], tf.float64)
        #accuracy_avg = tf.Print(accuracy_avg, [[start_true, end_true], [pr_start_idx, pr_end_idx], accuracy_avg], 'True and Predicted: ')


#avg_accuracy_ph = tf.placeholder(tf.float32, ())
tf.summary.scalar('avg_accuracy_test',  accuracy_avg, ["TEST_STAT"])
tf.summary.scalar('avg_accuracy_train', accuracy_avg, ["TRAIN_STAT"])


with tf.name_scope('Train'):
    train_step = optimizer.minimize(sum_loss)


In [ ]:
#=========== Training ==================

#dataset = ds.getDataset(["./train_train_task_b.csv"], max_sequence_length)
#iterator = dataset.make_one_shot_iterator()
#next_element = iterator.get_next()

#dataset_validation = ds.getDataset(["./test_dataset_160.csv"], max_sequence_length)
#iterator_valid = dataset_validation.make_one_shot_iterator()
#next_element_valid = iterator_valid.get_next()

summary_op_train = tf.summary.merge_all("TRAIN_STAT")
summary_op_test = tf.summary.merge_all("TEST_STAT")
summary_op = tf.summary.merge_all()

init = tf.global_variables_initializer()
DATASET_LENGTH = 50
with tf.Session() as sess:
    sess.run(init)
    h_param_str = utils.make_h_param_string(FLAGS.learning_rate, FLAGS.lstm_size, max_sequence_length, FLAGS.maxout_pooling_size)
    writer = tf.summary.FileWriter(FLAGS.log_path + "/3-" + h_param_str, sess.graph)
    with open('test_dataset_160.csv') as file_test:
        for epoch_ in range(100 or max_epoch):
            step_ = 0;
            with open('train_dataset_160.csv') as file_train:
                while True:
                    feed_dict = tr.processLineBatch(file_train, batch_size, 
                                                    max_sequence_length, max_question_length, 
                                                    question_ph, document_ph, dropout_rate_ph,
                                                    doc_len_ph, que_len_ph, start_true, end_true,
                                                    0.5)
                    if feed_dict is None: break
                    tr.trainStep(sess, 
                                 feed_dict, 
                                 writer, 
                                 train_step, sum_loss, accuracy_avg, summary_op, summary_op_train, 
                                 epoch_ * DATASET_LENGTH + step_, profiling=False)
                    step_+= 1
                    if (step_ >= DATASET_LENGTH): break;
                        
            print('Epoch', epoch_, 'completed')
            test_params = tr.processLineBatch(file_test, batch_size, max_sequence_length, max_question_length, 
                                              question_ph, document_ph, dropout_rate_ph,
                                              doc_len_ph, que_len_ph, start_true, end_true,
                                              1)
            tr.accuracyTest(sess, test_params, writer, 
                            accuracy_avg, summary_op, summary_op_test, pr_start_idx, pr_end_idx, epoch_)
        print('End')
            

--- Train step  1.7842800617218018 seconds ---
--- Train step  1.1330835819244385 seconds ---
--- Train step  1.1245825290679932 seconds ---
--- Train step  1.1256258487701416 seconds ---
--- Train step  1.1217820644378662 seconds ---
--- Train step  1.1150367259979248 seconds ---
--- Train step  1.118943691253662 seconds ---
--- Train step  1.1196386814117432 seconds ---
--- Train step  1.1066820621490479 seconds ---
--- Train step  1.1129786968231201 seconds ---
--- Train step  1.1182787418365479 seconds ---
--- Train step  1.1206793785095215 seconds ---
--- Train step  1.1260476112365723 seconds ---
--- Train step  1.1353905200958252 seconds ---
--- Train step  1.1323795318603516 seconds ---
--- Train step  1.131277084350586 seconds ---
--- Train step  1.1240196228027344 seconds ---
--- Train step  1.1466801166534424 seconds ---
--- Train step  1.1400713920593262 seconds ---
--- Train step  1.1345891952514648 seconds ---
--- Train step  1.1457538604736328 seconds ---
--- Train step 

--- Train step  1.1408815383911133 seconds ---
--- Train step  1.1491622924804688 seconds ---
--- Train step  1.1536245346069336 seconds ---
--- Train step  1.1533987522125244 seconds ---
--- Train step  1.149285078048706 seconds ---
--- Train step  1.1422491073608398 seconds ---
--- Train step  1.1329913139343262 seconds ---
--- Train step  1.1463334560394287 seconds ---
--- Train step  1.1457445621490479 seconds ---
--- Train step  1.1466104984283447 seconds ---
--- Train step  1.129744529724121 seconds ---
--- Train step  1.1550135612487793 seconds ---
--- Train step  1.1560099124908447 seconds ---
--- Train step  1.1312267780303955 seconds ---
--- Train step  1.134155035018921 seconds ---
--- Train step  1.1353702545166016 seconds ---
--- Train step  1.139340877532959 seconds ---
--- Train step  1.1371135711669922 seconds ---
--- Train step  1.1352295875549316 seconds ---
--- Train step  1.1272070407867432 seconds ---
--- Train step  1.143691062927246 seconds ---
--- Train step  1.

--- Train step  1.1044316291809082 seconds ---
--- Train step  1.1177809238433838 seconds ---
--- Train step  1.0973966121673584 seconds ---
--- Train step  1.1071984767913818 seconds ---
Epoch 6 completed
AVG accuracy 0.025
--- Train step  1.1196846961975098 seconds ---
--- Train step  1.106813669204712 seconds ---
--- Train step  1.1286475658416748 seconds ---
--- Train step  1.1087315082550049 seconds ---
--- Train step  1.1058483123779297 seconds ---
--- Train step  1.1278390884399414 seconds ---
--- Train step  1.128431797027588 seconds ---
--- Train step  1.1133701801300049 seconds ---
--- Train step  1.1135268211364746 seconds ---
--- Train step  1.1273152828216553 seconds ---
--- Train step  1.120340347290039 seconds ---
--- Train step  1.1136064529418945 seconds ---
--- Train step  1.126103162765503 seconds ---
--- Train step  1.109438180923462 seconds ---
--- Train step  1.1275520324707031 seconds ---
--- Train step  1.1173577308654785 seconds ---
--- Train step  1.1200432777

--- Train step  1.095870018005371 seconds ---
--- Train step  1.0956852436065674 seconds ---
--- Train step  1.0984861850738525 seconds ---
--- Train step  1.0893242359161377 seconds ---
--- Train step  1.0948491096496582 seconds ---
--- Train step  1.101252555847168 seconds ---
--- Train step  1.094069004058838 seconds ---
--- Train step  1.0863049030303955 seconds ---
--- Train step  1.1187291145324707 seconds ---
--- Train step  1.1002767086029053 seconds ---
--- Train step  1.0963466167449951 seconds ---
--- Train step  1.1089856624603271 seconds ---
--- Train step  1.114152193069458 seconds ---
--- Train step  1.110525369644165 seconds ---
--- Train step  1.1052947044372559 seconds ---
--- Train step  1.1094410419464111 seconds ---
--- Train step  1.1033813953399658 seconds ---
--- Train step  1.1029503345489502 seconds ---
--- Train step  1.0914928913116455 seconds ---
--- Train step  1.0886366367340088 seconds ---
--- Train step  1.1012682914733887 seconds ---
--- Train step  1.

--- Train step  1.1095397472381592 seconds ---
--- Train step  1.116333246231079 seconds ---
--- Train step  1.1249291896820068 seconds ---
--- Train step  1.111577033996582 seconds ---
--- Train step  1.1061193943023682 seconds ---
--- Train step  1.1043343544006348 seconds ---
--- Train step  1.102179765701294 seconds ---
--- Train step  1.102203607559204 seconds ---
--- Train step  1.1095471382141113 seconds ---
Epoch 13 completed
AVG accuracy 0.0232732732733
--- Train step  1.112419605255127 seconds ---
--- Train step  1.115851879119873 seconds ---
--- Train step  1.1157495975494385 seconds ---
--- Train step  1.107889175415039 seconds ---
--- Train step  1.1015026569366455 seconds ---
--- Train step  1.1057748794555664 seconds ---
--- Train step  1.1044464111328125 seconds ---
--- Train step  1.0856671333312988 seconds ---
--- Train step  1.1068923473358154 seconds ---
--- Train step  1.098658800125122 seconds ---
--- Train step  1.1021428108215332 seconds ---
--- Train step  1.09

...

--- Train step  1.1000795364379883 seconds ---
--- Train step  1.1017603874206543 seconds ---
--- Train step  1.1051068305969238 seconds ---
--- Train step  1.1110386848449707 seconds ---
--- Train step  1.1134271621704102 seconds ---
--- Train step  1.1119589805603027 seconds ---
--- Train step  1.1015093326568604 seconds ---
--- Train step  1.1004314422607422 seconds ---
--- Train step  1.1548128128051758 seconds ---
--- Train step  1.1205902099609375 seconds ---
--- Train step  1.1129522323608398 seconds ---
--- Train step  1.1087470054626465 seconds ---
--- Train step  1.1017279624938965 seconds ---
--- Train step  1.108177900314331 seconds ---
--- Train step  1.1059539318084717 seconds ---
--- Train step  1.1041626930236816 seconds ---
--- Train step  1.09598970413208 seconds ---
--- Train step  1.1004128456115723 seconds ---
--- Train step  1.0990066528320312 seconds ---
--- Train step  1.0968308448791504 seconds ---
--- Train step  1.09615159034729 seconds ---
--- Train step  1.

--- Train step  1.1001982688903809 seconds ---
--- Train step  1.0995492935180664 seconds ---
--- Train step  1.1074304580688477 seconds ---
--- Train step  1.114870309829712 seconds ---
Epoch 39 completed
AVG accuracy 0.0507936507937
--- Train step  1.1213159561157227 seconds ---
--- Train step  1.1322777271270752 seconds ---
--- Train step  1.1933386325836182 seconds ---
--- Train step  1.1156790256500244 seconds ---
--- Train step  1.1094005107879639 seconds ---
--- Train step  1.1071195602416992 seconds ---
--- Train step  1.0969178676605225 seconds ---
--- Train step  1.1001181602478027 seconds ---
--- Train step  1.0989809036254883 seconds ---
--- Train step  1.0964744091033936 seconds ---
--- Train step  1.1169886589050293 seconds ---
--- Train step  1.1095755100250244 seconds ---
--- Train step  1.0969092845916748 seconds ---
--- Train step  1.1017343997955322 seconds ---
--- Train step  1.1169333457946777 seconds ---
--- Train step  1.1153714656829834 seconds ---
--- Train ste

--- Train step  1.102668046951294 seconds ---
--- Train step  1.1290397644042969 seconds ---
--- Train step  1.181122064590454 seconds ---
--- Train step  1.1281826496124268 seconds ---
--- Train step  1.1250276565551758 seconds ---
--- Train step  1.1120569705963135 seconds ---
--- Train step  1.1152117252349854 seconds ---
--- Train step  1.132350206375122 seconds ---
--- Train step  1.110733985900879 seconds ---
--- Train step  1.1114633083343506 seconds ---
--- Train step  1.105428695678711 seconds ---
--- Train step  1.1042513847351074 seconds ---
--- Train step  1.1124205589294434 seconds ---
--- Train step  1.1179206371307373 seconds ---
--- Train step  1.1202561855316162 seconds ---
--- Train step  1.1151514053344727 seconds ---
--- Train step  1.103874683380127 seconds ---
--- Train step  1.1022207736968994 seconds ---
--- Train step  1.112602710723877 seconds ---
--- Train step  1.1164140701293945 seconds ---
--- Train step  1.1152045726776123 seconds ---
--- Train step  1.10

--- Train step  1.1167261600494385 seconds ---
--- Train step  1.1298003196716309 seconds ---
--- Train step  1.1281890869140625 seconds ---
--- Train step  1.1230144500732422 seconds ---
--- Train step  1.1177482604980469 seconds ---
--- Train step  1.1167595386505127 seconds ---
--- Train step  1.104973316192627 seconds ---
--- Train step  1.1087489128112793 seconds ---
--- Train step  1.1242668628692627 seconds ---
Epoch 46 completed
AVG accuracy 0.0240067971405
--- Train step  1.1228435039520264 seconds ---
--- Train step  1.1032259464263916 seconds ---
--- Train step  1.1004672050476074 seconds ---
--- Train step  1.1194627285003662 seconds ---
--- Train step  1.098738670349121 seconds ---
--- Train step  1.1085901260375977 seconds ---
--- Train step  1.1097218990325928 seconds ---
--- Train step  1.1202380657196045 seconds ---
--- Train step  1.1072211265563965 seconds ---
--- Train step  1.1034941673278809 seconds ---
--- Train step  1.1108307838439941 seconds ---
--- Train step

--- Train step  1.1363282203674316 seconds ---
--- Train step  1.1119332313537598 seconds ---
--- Train step  1.1211767196655273 seconds ---
--- Train step  1.1092984676361084 seconds ---
--- Train step  1.1144516468048096 seconds ---
--- Train step  1.0988588333129883 seconds ---
--- Train step  1.0996532440185547 seconds ---
